In [1]:
# Import necessary libraries
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
#from tqdm import tqdm  # Import tqdm for progress bar


# Define a function to load data
def load_data(base_path):
    datasets = {}
    # List of filenames to load
    filenames = ['target_holdout', 'medclms_holdout_cleaned9.29', 'rxclms_holdout_cleaned9.29']

    for filename in filenames:
        dataset = pd.read_csv(f"{base_path}/{filename}.csv", low_memory=False, encoding='ISO-8859-1')
        dataset.fillna('N/A', inplace=True)
        datasets[filename] = dataset

    return datasets


In [2]:


# Define the base path where the data is located
base_path = "/Users/nathanzlomke/Downloads"

# Load the datasets using the defined function
datasets = load_data(base_path)


/var/folders/4s/dws9r14s699_2rwx3m1tvp5m0000gn/T/ipykernel_8499/867876001.py:19: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'N/A' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  dataset.fillna('N/A', inplace=True)
/var/folders/4s/dws9r14s699_2rwx3m1tvp5m0000gn/T/ipykernel_8499/867876001.py:19: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'N/A' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  dataset.fillna('N/A', inplace=True)


In [3]:
datasets['rxclms_holdout_cleaned9.29'].rename(columns = {'ï»¿therapy_id':'therapy_id'}, inplace = 'True')

In [4]:
datasets['rxclms_holdout_cleaned9.29'].head()

,therapy_id,document_key,ndc_id,service_date,process_date,Prescription_Filled_Duration,RX_Process_Duration,pay_day_supply_cnt,rx_cost,tot_drug_cost_accum_amt,...,strength_meas,metric_strength,specialty_ind,clm_type,ddi_ind,anticoag_ind,diarrhea_treat_ind,nausea_treat_ind,seizure_treat_ind,RX_Groupings
0,1016317684-TAGRISSO-1,A195005006341001,228253950,4/10/19,4/11/19,-89,1,30,13.70,4725.09,...,MG-100MG,25.0,NONSPCL,rx,No,No,No,No,No,CNS
1,1084085456-TAGRISSO-1,A195044963981001,29300024210,4/14/19,4/15/19,-78,1,30,1.33,1228.73,...,MG,5.0,NONSPCL,rx,No,No,No,No,No,Cardio
2,1016343065-TAGRISSO-1,A195078392391001,65862019899,4/17/19,4/18/19,-77,1,90,5.36,6.16,...,MG,100.0,NONSPCL,rx,No,No,No,No,No,CNS
3,1096559829-TAGRISSO-1,A195157798591001,50242006401,4/25/19,4/26/19,-85,1,30,9043.00,27129.00,...,MG,150.0,SPCL,rx,No,No,No,No,No,Cancer
4,1016317684-TAGRISSO-1,A195223757081001,60505082901,5/2/19,5/3/19,-67,1,30,6.10,4738.79,...,MCG/ACT,50.0,NONSPCL,rx,No,No,No,No,No,Respiratory


In [12]:

# Merge all datasets together on 'therapy_id' as the primary key using an inner join
merged_data = datasets['target_holdout'].merge(datasets['rxclms_holdout_cleaned9.29'], on='therapy_id', how='left')


In [13]:

merged_data = merged_data.merge(datasets['medclms_holdout_cleaned9.29'], on='therapy_id', how='left')


In [14]:
merged_data.columns

Index(['therapy_id', 'medclm_key', 'clm_unique_key', 'primary_diag_cd',
       'visit_date', 'Visit_Duration', 'diag_cd2', 'diag_cd3', 'diag_cd4',
       'diag_cd5', 'diag_cd6', 'diag_cd7', 'diag_cd8', 'diag_cd9',
       'process_date_x', 'MedProcess_Duration', 'reversal_ind', 'pot',
       'util_cat', 'hedis_pot', 'clm_type_x', 'ade_diagnosis',
       'seizure_diagnosis', 'pain_diagnosis', 'fatigue_diagnosis',
       'nausea_diagnosis', 'hyperglycemia_diagnosis', 'constipation_diagnosis',
       'diarrhea_diagnosis', 'PD_C00ÐD49_FLG', 'PD_D50ÐD89_FLG',
       'PD_E00ÐE90_FLG', 'PD_G00ÐG99_FLG', 'PD_I00ÐI99_FLG', 'PD_J00ÐJ99_FLG',
       'PD_M00ÐM99_FLG', 'PD_R00ÐR99_FLG', 'PD_Z00ÐZ99_FLG', 'PD_Other_FLG',
       'NPD_C00ÐD49', 'NPD_D50ÐD89', 'NPD_E00ÐE90', 'NPD_G00ÐG99',
       'NPD_I00ÐI99', 'NPD_J00ÐJ99', 'NPD_K00ÐK93', 'NPD_M00ÐM99',
       'NPD_R00ÐR99', 'NPD_Z00ÐZ99', 'NPD_Other', 'NPD_SUM', 'document_key',
       'ndc_id', 'service_date', 'process_date_y',
       'Prescription_F

In [11]:

#There is no target variable for holdout data
'''# Convert 'Yes' to 1 and 'No' to 0 in the 'tgt_ade_dc_ind' column
merged_data['tgt_ade_dc_ind'] = merged_data['tgt_ade_dc_ind'].replace({'Yes': 1, 'No': 0})

# Print statistics about the column 'tgt_ade_dc_ind'
print("Statistics about 'tgt_ade_dc_ind' column:")
print(merged_data['tgt_ade_dc_ind'].describe())
'''


KeyError: 'tgt_ade_dc_ind'

In [32]:
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3878636 entries, 0 to 3878635
Data columns (total 91 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   therapy_id                    object 
 1   medclm_key                    int64  
 2   clm_unique_key                float64
 3   primary_diag_cd               object 
 4   visit_date                    object 
 5   Visit_Duration                object 
 6   diag_cd2                      object 
 7   diag_cd3                      object 
 8   diag_cd4                      object 
 9   diag_cd5                      object 
 10  diag_cd6                      object 
 11  diag_cd7                      object 
 12  diag_cd8                      object 
 13  diag_cd9                      object 
 14  process_date_x                object 
 15  MedProcess_Duration           int64  
 16  reversal_iNod_x               object 
 17  pot                           object 
 18  util_cat              

In [33]:
merged_data.head()

,therapy_id,medclm_key,clm_unique_key,primary_diag_cd,visit_date,Visit_Duration,diag_cd2,diag_cd3,diag_cd4,diag_cd5,...,id,therapy_start_date,therapy_end_date,Date Duration,tgt_ade_dc_ind,race_cd,est_age,sex_cd,cms_disabled_ind,cms_low_income_ind
0,1066310426-TAGRISSO-1,35908472910,6.490000e+17,I70292,9/10/19,-128.0,I771,I739,R9431,Z7982,...,1066310426,1/16/20,2/15/20,30,0,White,74.0,M,No,No
1,1066310426-TAGRISSO-1,35908472910,6.490000e+17,I70292,9/10/19,-128.0,I771,I739,R9431,Z7982,...,1066310426,1/16/20,2/15/20,30,0,White,74.0,M,No,No
2,1066310426-TAGRISSO-1,35908472910,6.490000e+17,I70292,9/10/19,-128.0,I771,I739,R9431,Z7982,...,1066310426,1/16/20,2/15/20,30,0,White,74.0,M,No,No
3,1066310426-TAGRISSO-1,35908472910,6.490000e+17,I70292,9/10/19,-128.0,I771,I739,R9431,Z7982,...,1066310426,1/16/20,2/15/20,30,0,White,74.0,M,No,No
4,1066310426-TAGRISSO-1,35908472910,6.490000e+17,I70292,9/10/19,-128.0,I771,I739,R9431,Z7982,...,1066310426,1/16/20,2/15/20,30,0,White,74.0,M,No,No


In [16]:
merged_data['therapy_id'].shape

(608123,)

In [15]:
merged_data.to_csv("CORRECTED_holdout_cleaned_humana_inner10-1.csv")